**Investigating Heart disease**

Heart disease is the leading cause of death across America. If we can understand it better using statistical models we can hopefully devise methods to detect the signs early. There are many different factors than can contribute to this disease, some of which may be missing from the dataset I am working with due mainly to the sheer number of them. However, using what we have can shed plenty of light on the nature of this problem. 

There are two goals in this project.The first is to get a better understanding of how some of these factors contribute to the overall scheme of things. The second is to build and validate models that will allow us to classify whether or not a patient has heart disease based on their symptoms.

Lets get started by first setting up our required libraries.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as tree
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Reading and Cleaning the Data**

Here is where we gather our data from our csv file. 

In [ ]:
data = pd.read_csv("../input/shuffled-heart/shuffled_heart.csv")
data.head()

One thing to note from an initial inspection is that some of the data is represented as dummy variables. The descriptions for these variables can be found in the dataset's documentation. 

Before we proceed, it is a good idea to check whether or not we need to clean up the data. The following code checks if there is any missing entries.

In [ ]:
data.isnull().values.any()

Thankfully all the columns are filled in; however, I want to make the data more intuitive to look at, thus we can replace the dummy variables using the dataset's documentation. Despite my desire to make the data more intuitive, it would be wise to keep all our dummy variables intact to avoid a headache when we are ready to build our models. Therefore, I will make a copy of the dataframe and fill it in with intuitive information. The 'intuitive' dataframe will be used for data exploration purposes, the original dataframe will be used for statistical analysis.

In [ ]:
intuitive = data.copy()

intuitive['cp'] = intuitive['cp'].replace(0,'No pain')
intuitive['cp'] = intuitive['cp'].replace(1,'Typical-agina')
intuitive['cp'] = intuitive['cp'].replace(2,'Atypical-agina')
intuitive['cp'] = intuitive['cp'].replace(3,'Non-agina')
intuitive['cp'] = intuitive['cp'].replace(4,'Asymptomatic')
intuitive['sex'] = intuitive['sex'].replace(1,'male')
intuitive['sex'] = intuitive['sex'].replace(0,'female')
intuitive['fbs'] = intuitive['fbs'].replace(0,'Under 120 mg/dl')
intuitive['fbs'] = intuitive['fbs'].replace(1,'Over 120 mg/dl')
intuitive ['restecg'] = intuitive['restecg'].replace(0,'Normal')
intuitive ['restecg'] = intuitive['restecg'].replace(1,'Abnormal')
intuitive['exang'] = intuitive['exang'].replace(1,'yes')
intuitive['exang'] = intuitive['exang'].replace(0,'no')
intuitive['target'] = intuitive['target'].replace(0,'No Heart Disease')
intuitive['target'] = intuitive['target'].replace(1,'Has Heart Disease')

intuitive.head()

Now this is better. We are ready to begin data exploration.

**Data exploration and visualization**

First up, I'll load in the data and construct a heatmap. This heatmap can help us visualize what characteristics of heart disease are correlated.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(data.corr(), annot = True, linewidths=.5, ax = ax).set_title("Correlation of Physical Characteristics")

Suprisingly there are no strikingly high correlations between any two variables. The two highest correlation values for target come from thalach( maximum heartrate achieved) and cp( chest pain type). I expected the top two to come from blood pressure and cholesterol levels as these factors directly relate to the heart yet these have relatively low correlations to diagnosis.  Let us see some statistics for chest pain type and maximum heart rate.

In [ ]:
data['cp'].mode()

In [ ]:
data['thalach'].describe()

Since the chest pain data was qualitative I was more interested in what was the most common chest pain type. It turns out typical agina was the most common type of chest pain. This type of pain can be identified by tightness in the chest area, shortness of breath and a sudden increase of perspiration. This chest pain is caused by blockages in the arteries resulting in the heart not getting enough oxygen. (Reference: https://www.harringtonhospital.org/typical-and-atypical-angina-what-to-look-for/)

Looking at the data for maximum heart rate, the average was 150 bpm. Using the formula well known formula MAX_HEARTRATE = (220 - AGE) we find that this average equates to the maximum heart rate for a 72 year old person. Now this begs the question: what does the age distribution look like?

In [ ]:
plt.figure(figsize=(11,4))
ax = sns.countplot(x = data['age']).set_title('Age distribuition of patients studied')

Now that is very interesting. Most people studied were in the age range of 50 through 60 and only a handful of patients were in their 70's. This can only mean that a majority of these patients exhibited a lower maximum heart rate than expected for their age. Why is this? Fortunately, that answer can be answered from our previous observation of chest pain. Remember, the most common type of chest pain is caused by semi clogged arteries. With less oxygen flowing into the heart, the heart will have a difficult time pumping out blood, hence a lower maximum heart rate. This also gives a physical explanation for why chest pain and heart rate had a noticible correlation on the heat map.

I now want to explore if a person's gender has any effect on anything. First lets look at the hand we're dealt.

In [ ]:
intuitive['sex'].value_counts().plot.bar(title = 'Genders of patients studied')

In [ ]:
intuitive['sex'].value_counts()

There were over twice the amount of male patients than there we female. This could be a bit problematic. If males exhibit different symptoms than women this would make this data biased towards males and it wouldnt fully capture what is going behind the scenes for females. 

Let us investigate the data gender specifically. First I will split the data by gender.  

In [ ]:
is_female = intuitive['sex'] == 'female'
is_male = intuitive['sex'] == 'male'

all_females = intuitive[is_female]
all_males = intuitive[is_male]

Now let us revisit the age distribution

In [ ]:
plt.figure(figsize=(11,4))
sns.distplot(all_females['age']).set_title('Age Distribution of Females')


In [ ]:
plt.figure(figsize = (11,4))
sns.distplot( all_males['age']).set_title('Age Distribution of Males')

From the plots we can see that males tend to report symptoms of heart disease at an earlier age than women. The distritbuition peaks in the late 50's for males and in the early 60's for females.  

Now let's observe if the chest pains differ between genders.

In [ ]:
sns.barplot(x = all_females['cp'].value_counts().index,y = all_females['cp'].value_counts(),palette = "rocket").set_title('Chest Pain Types Reported By Females')

In [ ]:
sns.barplot(x = all_males['cp'].value_counts().index,y = data['cp'].value_counts(),palette = "deep").set_title('Chest Pain Types Reported By Males')

Now isn't that strange? No chest pain is the most common occurance. Yet, the females and males both exhibit the same distribution of pain types reported. 

In [ ]:
sns.set(style = "whitegrid")
sns.violinplot(x = intuitive['sex'], y = intuitive['thalach']).set_title("Maximum Heart Rate")


The median values and probability densities for the median for both genders are nearly the same, which is to be expected. One thing to notice though is that males only have one peak at the median. Females, on the other hand, have a second peak near the 125 bpm range. 
The last two pieces of analysis confirm there is a noticble difference between male and female symptoms. It would be wise to re-analyze the correlations but in a gender specific way.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(data[is_female].corr(), annot = True, linewidths=.5, ax = ax).set_title("Correlation For Females")

In [ ]:
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(data[is_male].corr(), annot = True, linewidths=.5, ax = ax).set_title("Correlation For Males")

Though not strikingly different from the original correlation heat map, we do notice a few things. Chest pain has a higher correlation to the females' diagnosis than males. Also, maximum heart rate was not the second most significant correlation for the diagnosis of females, it was slope which measures peak performance of exercise. What does this mean? Typical-agina pain is often induced through strenuous activity, including excercise. Since our data shows that females are extremely likely to feel this kind of pain we can infer that older females who have a underlying heart disease exercise put themselves at risk of inducing chest pain.

From our analysis we were able to make some very interesting observations. Now that we're familiar enough with the data, it's time to model it.

**Preparing for modeling**

When I first downloaded the dataset it was too neatly sorted. Specifically, all the rows with the same diagnosis had been grouped together. Thus I shuffled the original dataset and saved it as a new .csv file. That shuffled file is the one we are working with now. 

**K-Fold Cross Validation** 

We are dealing with a binary classification problem that is not neccisarily well separated. If I had to guess, I'd say our best bet would be logistic regression but lets quantify that theory against other popular classification models. K-fold cross validation can help out with that. In this case we will be using 5-fold cross validation.

In [ ]:
cols = ['age','sex','cp','trestbps','chol','fbs','restecg','exang','oldpeak','slope','ca','thal']
goal = ['target']

logic_score = cross_val_score(LogisticRegression(), data[cols], data[goal], cv = 5,scoring = 'accuracy')
l1_logic = cross_val_score(LogisticRegression(penalty = 'l1'), data[cols], data[goal], cv = 5,scoring = 'accuracy')
lda_score = cross_val_score(LinearDiscriminantAnalysis(solver = 'svd'),data[cols],data[goal],cv = 5, scoring = 'accuracy')
svc_lin_score = cross_val_score(SVC(kernel = 'linear', C = 1), data[cols], data[goal], cv = 5,scoring = 'accuracy')
tree_score = cross_val_score(tree(random_state = 0), data[cols], data[goal], cv = 5,scoring = 'accuracy')



In [ ]:
print('L2 Logistic Regression Score', logic_score.mean())
print('L1 Logistic Regression Score:', l1_logic.mean())
print('Linear Discriminant Analysis Score:', lda_score.mean())
print('Linear Support Vector Machine Score', svc_lin_score.mean())
print('Descision Tree Score', tree_score.mean())

Logistic regresion, LDA and SVM seem to be the best fit for our model. Let's tweak these models to see if we can get a better score.

In [ ]:
plt.figure(figsize = (11,4))
plt.plot(logic_score, label = 'L2 Logistic Regression')
plt.plot(l1_logic,label = 'L1 Logistic Regression')
plt.plot(lda_score, label = 'Linear Discriminant Analysis')
plt.plot(svc_lin_score, label = 'Support Vector Classification')
plt.legend()

Our best accuracies came from linear discriminant analysis and support vector classification both having an accuracy of 82%. It's time to experiment and tweak our features to see if we can improve the model.

**Transforming using LDA**

Linear Discriminant Analysis aims to maximize seperability in our data while reducing the dimensions. Perhaps transforming the data using LDA and then applying a model could increase the accuracy of predictions. 

In [ ]:
train,test = np.split(data,[data['age'].count() // 2],axis = 0)
X_train = train[cols]
y_train = train[goal]
X_test = test[cols]
y_test = test[goal]

lda = LinearDiscriminantAnalysis()
lda.fit(X_train,y_train)
transformed = lda.transform(data[cols])



In [ ]:
transformed_df = pd.concat([pd.DataFrame(transformed),data['target']] ,axis = 1)


In [ ]:
plt.plot(transformed_df[0])

In [ ]:
lda_logic_score = cross_val_score(LogisticRegression(), transformed, data[goal], cv = 5,scoring = 'accuracy')

In [ ]:
lda_logic_score.mean()

In [ ]:
plt.plot(lda_logic_score)

In [ ]:
lda_svc = cross_val_score(SVC(kernel = 'linear', C = 1), transformed, data[goal], cv = 5,scoring = 'accuracy')

In [ ]:
plt.plot(lda_svc)

In [ ]:
lda_svc.mean()

We are definitely moving in the right direction! Logistic regression's accuracy increased by 4% and while support vector classification increased by 2%. 

**RFE**

In [ ]:
#from sklearn.feature_selection import RFE
logic_scores = []
for x in range(1,12):
    logistic_rfe = RFE(LogisticRegression(), x, step=1)
    logistic_rfe.fit(X_train,y_train)
    logic_scores.append(logistic_rfe.score(X_test,y_test))
logic_scores

In [ ]:
plt.plot(logic_scores)

**Scaling**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
df_scaled = pd.DataFrame(data_scaled)

In [ ]:
strain,stest = np.split(df_scaled,[df_scaled[1].count() // 2],axis = 0)
s_cols = [0,1,2,3,4,5,6,7,8,9,10,11,12]
s_goal = [13]
XScale_train = strain[s_cols]
yScale_train = strain[s_goal]
XScale_test = stest[s_cols]
yScale_test = stest[s_goal]
scaled_logic_score = cross_val_score(LogisticRegression(), df_scaled[s_cols], df_scaled[s_goal], cv = 5,scoring = 'accuracy')

In [ ]:
scaled_logic_score.mean()

In [ ]:
plt.plot(scaled_logic_score)

**Separation by gender**

From our data exploration we know that the males out numbered the females. Perhaps separating the data by gender and making seperate models for each gender can improve our accuracy.

In [ ]:
fem = data[is_female]
males = data[is_male]

female_logic_score = cross_val_score(LogisticRegression(), fem[cols], fem[goal], cv = 5,scoring = 'accuracy')
male_logic_score = cross_val_score(LogisticRegression(), males[cols], males[goal], cv = 5,scoring = 'accuracy')

In [ ]:
plt.plot(female_logic_score)

In [ ]:
plt.plot(male_logic_score)

In [ ]:
print("Female Score:", female_logic_score.mean())
print("Male Score:", male_logic_score.mean())

We see a significant increase in the female model but the male model took a significant hit in accuracy. 

**PCA**

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principle_components = pca.fit_transform(data[cols])
pca_cols = ['principle component 1', 'principle component 2']
pca_df = pd.DataFrame(data = principle_components, columns = pca_cols)
pca_df = pd.concat([pca_df, data['target']], axis = 1)

In [ ]:
pca_logic = cross_val_score(LogisticRegression(), pca_df[pca_cols], data[goal], cv = 5,scoring = 'accuracy')
pca_svc = cross_val_score(SVC(kernel = 'linear', C = 1), pca_df[pca_cols], pca_df['target'], cv = 5,scoring = 'accuracy')

In [ ]:
print('PCA Logistic:', pca_logic.mean())
print('PCA SVC:', pca_svc.mean())

**Ensemble Methods**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
transformed_df = pd.DataFrame(transformed)
transformed_df = pd.concat([transformed_df,data['target']], axis = 1)
tr_train,tr_test = np.split(transformed_df,[transformed_df[0].count() // 2],axis = 0)
XT_train = tr_train[0]
yt_train = tr_train['target']
XT_test = tr_test[0]
yt_test = tr_test['target']
rfc = RandomForestClassifier()
rfc.fit(XT_train.values.reshape(-1,1),yt_train.values.reshape(-1,1))
rfc.score(XT_test.values.reshape(-1,1),yt_test.values.reshape(-1,1))


In [ ]:
rfc.fit(X_train,y_train)
rfc.score(X_test,y_test)

The xgboost library can help us develop a more accurate model using gradient boosting. 

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb = XGBRegressor(n_estimators = 100)
xgb.fit(X_train ,y= y_train)
xgb_predict = xgb.predict(X_test)
xgb_mse = mean_squared_error(xgb_predict,y_test)

rounded = []
for x in range(len(xgb_predict)):
    rounded.append(int(xgb_predict[x].round()))
r = np.asarray(rounded)
te = y_test.as_matrix().tolist()
acc = [0,0]
for x in range(len(r)):
    acc[1]+= 1
    if(te[x][0] == rounded[x]):
        acc[0] += 1
accuracy = acc[0] / acc[1] * 100
print('Gradient boosting MSE:',xgb_mse)
print('Accuracy:',accuracy )

In [ ]:
train,test = np.split(transformed_df,[data['age'].count() // 2],axis = 0)
XT_train = train[0]
yt_train = train['target']
XT_test = test[0]
yt_test = test['target']

txgb = XGBRegressor(n_estimators = 100)
txgb.fit(XT_train.values.reshape(-1,1),yt_train.values.reshape(-1,1))
txgb_predict = txgb.predict(XT_test.values.reshape(-1,1))
#mean_squared_error(txgb_predict,yt_test)
rounded = []
for x in range(len(txgb_predict)):
    rounded.append(int(txgb_predict[x].round()))
r = np.asarray(rounded)
acc = [0,0]
yte = yt_test.values.reshape(-1,1)
for x in range(len(r)):
    acc[1]+= 1
    if(yte[x][0] == rounded[x]):
        acc[0] += 1
accuracy = acc[0] / acc[1] * 100
accuracy
#yt_test.values.reshape(-1,1)[0][0]